<a href="https://colab.research.google.com/github/antoreep-jana/YouTube_Code_Repositories/blob/main/Weekly%20Sessions/Weekly_Session__12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tensorflow Model Sub-Classing

In [ ]:
import tensorflow as tf

In [ ]:
class CustomModel(tf.keras.Model):

  def __init__(self, num_classes = 1000, verbose = True, bn = True):

    super(CustomModel, self).__init__()

    self.conv1 = tf.keras.layers.Conv2D(32, (2,2), strides = 1, activation ='relu')
    self.max1 = tf.keras.layers.MaxPool2D((2,2))
    
    if bn:
      self.bn1 = tf.keras.layers.BatchNormalization()
    
      if verbose:
        print("Batch Normalization Layer addded!")
    else:
      if verbose:
        print("Batch Normalization Layer not added! :(")


    self.conv2 = tf.keras.layers.Conv2D(64, (2,2), strides = 1, activation = 'relu')
    self.max2 = tf.keras.layers.MaxPool2D((2,2))

    self.flatten = tf.keras.layers.Flatten()
    self.dense = tf.keras.layers.Dense(num_classes, activation = 'softmax') 


  def call(self, input_vector):
    
    x = self.conv1(input_vector)
    x = self.max1(x)
  
    
    x = self.bn1(x)
  
    x = self.conv2(x)
    x = self.max2(x)

    return self.dense(self.flatten(x))

  def fit(self):
    pass 
    

  def evaluate(self):
    pass 



  def summary(self):

    X = tf.keras.layers.Input(shape = (224,224,3))
    model = tf.keras.Model(inputs = [X], outputs = self.call(X))
    return model.summary()


In [ ]:
model = CustomModel(1000, verbose = True, bn = False)

Batch Normalization Layer not added! :(


In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.build(input_shape = (None, 224,224,3))

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 223, 223, 32)      416       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 111, 111, 32)      128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 110, 110, 64)      8256      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 55, 55, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 193600)            0     

Custom Layers using TF Keras

Step1. Define the init() with layer parameter values <br>
Step2. Define the build() to build your layer with kernel matrix & bias<br>
Step3. Define the call() to define the forward pass in the layer. Usually matmul of inputs & weights (kernel) + bias

In [ ]:
class CustomLayer(tf.keras.layers.Layer):

  def __init__(self, units = 128, activation = 'relu', initializer = 'he_normal', **kwargs):

    #self.name = name
    super(CustomLayer, self).__init__(**kwargs)
    self.units = units 
    self.activation = tf.keras.activations.relu
    self.initializer = tf.keras.initializers.HeNormal()



  def build(self, input_shape):
    
    self.w = self.add_weight(shape = (input_shape[-1], self.units), initializer = self.initializer, trainable = True)
  

    self.b = self.add_weight(shape = (self.units,), initializer = self.initializer, trainable = True)


  def call(self, input_tensor):
    

    # WX + B
    result = tf.matmul(input_tensor, self.w) + self.b


    # act(WX + B)
    if self.activation:
      result = self.activation(result)

    return result

  def get_config(self):
   
    # Step 1 : Get the configuration from the base layer 

    # Step 2 : make a dictionary of what all you want to save and update

    # return the dictionary 

    config = super(CustomLayer, self).get_config()

    config.update({'units' : self.units, 'initializer' : self.initializer})

    return config



In [ ]:
layer = CustomLayer()

In [ ]:
ini = tf.random_uniform_initializer()
tensor = ini((4,3)) 

In [ ]:
layer(tensor)

<tf.Tensor: shape=(4, 128), dtype=float32, numpy=
array([[0.        , 0.20251326, 0.        , 0.02407259, 0.13172732,
        0.10345115, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00918717, 0.06233312, 0.20667541, 0.17371081,
        0.12666488, 0.        , 0.        , 0.21836826, 0.        ,
        0.        , 0.        , 0.1898109 , 0.        , 0.        ,
        0.13425201, 0.12860796, 0.        , 0.        , 0.        ,
        0.        , 0.01862221, 0.06088087, 0.        , 0.        ,
        0.13225123, 0.00872555, 0.        , 0.0536253 , 0.        ,
        0.        , 0.        , 0.05839918, 0.        , 0.        ,
        0.        , 0.03621362, 0.        , 0.10460845, 0.07518675,
        0.        , 0.15317063, 0.05590467, 0.07958807, 0.07012384,
        0.11997245, 0.        , 0.01304003, 0.        , 0.        ,
        0.08785532, 0.        , 0.16330393, 0.        , 0.06547214,
        0.        , 0.0909026 , 0.        , 0.06217775, 0.        

In [ ]:
layer.weights

[<tf.Variable 'custom_layer_9/Variable:0' shape=(3, 128) dtype=float32, numpy=
 array([[ 0.8645956 , -0.30999225, -0.16832232, -0.01756698, -0.51605386,
         -1.6395372 , -0.893023  ,  0.21165106, -1.073722  , -1.058385  ,
         -0.54467696, -0.45273787,  0.4752862 ,  0.91914344,  0.6740766 ,
          0.12781833,  0.65386605, -1.1760008 , -0.38099346,  0.16845681,
         -1.3028272 ,  0.32806522,  0.75829023, -0.7189064 , -1.3374047 ,
         -1.133761  ,  1.1902441 , -1.1107984 ,  1.10848   , -0.21750028,
          0.10185356,  0.97963804,  0.36332726,  0.20178069, -0.8693004 ,
         -1.1042235 , -1.1777407 ,  0.85079634, -0.4152919 ,  0.16369846,
          0.54429656, -0.38443947, -0.2644578 ,  0.6874285 , -0.6393806 ,
          0.3571887 ,  0.95451653, -0.54029256,  1.1571552 , -1.3102556 ,
          0.42994004,  1.0364927 ,  0.9854941 , -0.2437649 , -0.05857303,
          1.5394187 ,  1.2889768 , -0.6634928 ,  0.07499183, -0.73037326,
         -0.7157861 , -0.29180473

In [ ]:
layer.b

<tf.Variable 'custom_layer_9/Variable:0' shape=(128,) dtype=float32, numpy=
array([-0.06360154,  0.21996878, -0.20522651,  0.05745121,  0.13929895,
        0.14708461, -0.15888903, -0.21654662, -0.06343951, -0.04338158,
       -0.04707361,  0.00337823,  0.0415279 ,  0.15158086,  0.15890676,
        0.10605574, -0.21750069, -0.16065194,  0.20785454, -0.1046757 ,
       -0.19611642, -0.11422905,  0.1449585 , -0.14282835, -0.02339963,
        0.17519759,  0.09656227, -0.03639217, -0.13515209, -0.08023664,
       -0.19427232, -0.02748103,  0.02796754, -0.27969566, -0.19130313,
        0.17057304,  0.04107957, -0.04829568,  0.05127375, -0.18688054,
       -0.13536948,  0.00225653,  0.07357666, -0.08950281, -0.00635243,
       -0.18380244,  0.00326852, -0.09529315,  0.08704999,  0.12417553,
       -0.01303769,  0.12970701,  0.03224787,  0.09633847,  0.06719056,
        0.06566972, -0.07006142,  0.03116766, -0.09126983, -0.20542198,
        0.12367238, -0.03527439,  0.17926699, -0.14955872,  

In [ ]:
layer.get_config()

{'dtype': 'float32',
 'initializer': <keras.initializers.initializers_v2.HeNormal at 0x7ff9c03c9e50>,
 'name': 'custom_layer_9',
 'trainable': True,
 'units': 128}

In [ ]:
layer1 = CustomLayer()

layer1.from_config(layer.get_config())

In [ ]:
class NestedLayers(tf.keras.layers.Layer):

  def __init__(self, units = 128):

    self.layer1 = CustomLayer1()
    self.layer2 = CustomLayer2()
  
  def build():
    pass 


  def call():
    pass 



NameError: ignored

## Custom Loss Functions

In [ ]:
class CustomLoss(tf.keras.losses.Loss):

  def __init__(self, y_pred, y_true):
    self.y_pred = y_pred
    self.y_true = y_true


  def call(self):
    result = tf.square(self.y_pred - self.y_true)

    return tf.reduce_mean(result, axis = -1)

In [ ]:
loss1 = CustomLoss(tf.ones((2, 2,)), tf.zeros((2, 2)))

In [ ]:
loss1.call()

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>

In [ ]:
def mean_sq_loss(y_pred, y_true):

  result = tf.square(y_pred - y_true)

  return tf.reduce_mean(result, axis = -1)